In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# kaggle https://www.kaggle.com/c/santander-customer-transaction-prediction

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense,Activation
from sklearn.model_selection import StratifiedKFold,cross_val_score
from keras.utils.generic_utils import get_custom_objects

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Checking to see the expected submission format
sample = pd.read_csv('../input/sample_submission.csv')

print(sample.head())

train = pd.read_csv('../input/train.csv')
print ("Train dataset has {} rows(samples) with {} columns(features) each.".format(*train.shape))

#test = pd.read_csv('test.csv', index_col='Id')
test = pd.read_csv('../input/test.csv')

print ("Test dataset has {} rows(samples) with {} columns(features) each.".format(*test.shape))

In [ ]:
import matplotlib.pyplot as plt

def get_categorical_variables(df,include_variables,exclude_variables,index_variable,label): 
    if include_variables is None: 
        include_variables=list(df.select_dtypes(include=[np.object,np.bool]).columns)
    if index_variable in include_variables:
          include_variables.remove(index_variable)

    if label is not None:
        if label in include_variables :
            if exclude_variables is None:
                 exclude_variables=label
            else:
                 exclude_variables=exclude_variables.append(label)

    if exclude_variables is None:
        include_variables_filtered=include_variables
    else:
        include_variables_filtered=[elem for elem in include_variables if elem not in exclude_variables] 
    return include_variables_filtered;

def get_numerical_variables(df,include_variables,exclude_variables,index_variable,label):
    if include_variables is None:
        include_variables=list(df.select_dtypes(exclude=[np.object,np.bool,np.datetime64]).columns)
    if index_variable in include_variables:
        include_variables.remove(index_variable)
    if label is not None:
        if label in include_variables :
            if exclude_variables is None:
                exclude_variables=label
            else:
                exclude_variables=exclude_variables.append(label)

    if exclude_variables is None:
          include_variables_filtered=include_variables
    else:
          include_variables_filtered=[elem for elem in include_variables if elem not in exclude_variables] 
    return include_variables_filtered;


In [ ]:
index_variable='ID_code'
label='target'
include_numerical_variables=None # if None then we select all except the  exclude_numerical_variables
include_categorical_variables=None # if None then we select all except the  exclude_categorical_variables
exclude_numerical_variables=None
exclude_categorical_variables=None
categorical_variables =get_categorical_variables(train,include_categorical_variables,exclude_categorical_variables,index_variable,label)
numerical_variables=get_numerical_variables(df=train,include_variables=None,exclude_variables=None,index_variable=index_variable,label=label)

display(('=====================Categorical Variables==================='))
display(categorical_variables)
  
display(('=====================Numerical Variables====================='))
display(numerical_variables)
display(('=====================Dependent Variable====================='))
display(label)

In [ ]:
from sklearn import preprocessing

def normalize_data(data):
    #normalize all the columns(features) so that all the values in the column lie between 0 and 1
    #this way each features will get equal preference regardless of their actual range
    n_data=pd.DataFrame()
    n_data = pd.DataFrame(preprocessing.normalize(data),columns=data.columns)
    return n_data

#def remove_outliers_by_value(data, column,value):
#    good_data = data.loc[~data.[column].isin(value)]
#    return good_data



In [ ]:
from sklearn.model_selection import train_test_split
id_code=train[index_variable]
y=train[label]
X=normalized_train=normalize_data(train[numerical_variables])
#X_trast=normalized_train=normalize_data(test[numerical_variables])
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.3)

In [ ]:
train.groupby('target').size()/train.shape[0]

In [ ]:
'''from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
from keras.layers import LeakyReLU,Activation
from keras.callbacks import TensorBoard
import sys

from keras import callbacks
from keras import backend as K

class MyLogger(callbacks.Callback):
    def __init__(self, n):
        self.n = n   # print loss & acc every n epochs

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.n == 0:
            curr_loss =logs.get('loss')
            curr_acc = logs.get('acc') * 100
            print("epoch = %4d  loss = %0.6f  acc = %0.2f%%" % (epoch, curr_loss, curr_acc))
    
    


def swish(x):
    return (K.sigmoid(x) * x)

my_logger = MyLogger(n=1)
#my_logger = TensorBoard(log_dir="logs/{}".format(time()))
#my_logger = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
#tensorboard --logdir output/Graph 
class_weight = {0: 1., 1: 9.}
space = {'choice': hp.choice('num_layers',
                    [ {'layers':'two', },
                    {'layers':'three',
                    'units3': hp.uniform('units3', 64,1024), 
                    'dropout3': hp.uniform('dropout3', .25,.75)}
                    ]),

            'units1': hp.uniform('units1', 64,1024),
            'units2': hp.uniform('units2', 64,1024),

            'dropout1': hp.uniform('dropout1', .25,.75),
            'dropout2': hp.uniform('dropout2',  .25,.75),

            'batch_size' : hp.uniform('batch_size', 28,128),

            'nb_epochs' :  100,
            'optimizer': hp.choice('optimizer',['adam']),
            'activation': hp.choice('activation',['LeakyReLU'])
        }

def f_nn(params):   
    from keras.models import Sequential
    from keras.layers.core import Dense, Dropout, Activation
    from keras.optimizers import Adadelta, Adam, rmsprop

    print ('Params testing: ', params)
    #print("shape testing: {}, {}".format(params['choice']['units3']))
    model = Sequential()
   
    model.add(Dense(units=round(params['units1']), input_dim = X_train.shape[1], kernel_initializer = "glorot_uniform")) 
    if params['activation']== 'LeakyReLU':
        model.add(LeakyReLU()) 
    else:
        model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout1']))

    model.add(Dense(units=round(params['units2']), kernel_initializer = "glorot_uniform")) 
    if params['activation']== 'LeakyReLU':
        model.add(LeakyReLU()) 
    else:
        model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout2']))

    if params['choice']['layers']== 'three':
        model.add(Dense(units=round(params['choice']['units3']), kernel_initializer = "glorot_uniform")) 
        if params['activation']== 'LeakyReLU':
            model.add(LeakyReLU()) 
        else:
            model.add(Activation(params['activation']))
        model.add(Dropout(params['choice']['dropout3']))    

    model.add(Dense(units=1, kernel_initializer='normal'))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=params['optimizer'],metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=round(params['nb_epochs']), batch_size=round(params['batch_size']),verbose = 0,callbacks=[my_logger], class_weight=class_weight)

    pred_auc =model.predict_proba(X_validate, batch_size = 128, verbose = 0)
    acc = roc_auc_score(y_validate, pred_auc)
    print('AUC:', acc)
    sys.stdout.flush() 
    return {'loss': -acc, 'status': STATUS_OK}


trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=50, trials=trials)
print ('best: ')
print (best)'''

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
from keras.layers import LeakyReLU,Activation
from keras.callbacks import TensorBoard
import sys

from keras import callbacks
from keras import backend as K

class MyLogger(callbacks.Callback):
    def __init__(self, n):
        self.n = n   # print loss & acc every n epochs

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.n == 0:
            curr_loss =logs.get('loss')
            curr_acc = logs.get('acc') * 100
            print("epoch = %4d  loss = %0.6f  acc = %0.2f%%" % (epoch, curr_loss, curr_acc))
    
    
#Params testing:                                                                  
params={'activation': 'LeakyReLU', 'batch_size': 121.49461530883102, 'choice': {'dropout3': 0.28556934532942824, 'layers': 'three', 'units3': 461.4279574939098}, 
 'dropout1': 0.5488421886416901, 'dropout2': 0.7395325011437675, 'nb_epochs': 100, 'optimizer': 'adam', 'units1': 686.4798766184405, 'units2': 450.82106283335355}

my_logger = MyLogger(n=1)
#my_logger = TensorBoard(log_dir="logs/{}".format(time()))
#my_logger = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
#tensorboard --logdir output/Graph 
class_weight = {0: 1., 1: 9.}
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta, Adam, rmsprop

print ('Params testing: ', params)
#print("shape testing: {}, {}".format(params['choice']['units3']))
model = Sequential()

model.add(Dense(units=round(params['units1']), input_dim = X_train.shape[1], kernel_initializer = "glorot_uniform")) 
model.add(LeakyReLU()) 
model.add(Dropout(params['dropout1']))

model.add(Dense(units=round(params['units2']), kernel_initializer = "glorot_uniform")) 
model.add(LeakyReLU()) 
model.add(Dropout(params['dropout2']))

model.add(Dense(units=round(params['choice']['units3']), kernel_initializer = "glorot_uniform")) 
model.add(LeakyReLU()) 
model.add(Dropout(params['choice']['dropout3']))    

model.add(Dense(units=1, kernel_initializer='normal'))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=params['optimizer'],metrics=['accuracy'])

model.fit(X_train, y_train, epochs=round(params['nb_epochs']), batch_size=round(params['batch_size']),verbose = 0,callbacks=[my_logger], class_weight=class_weight)

pred_auc =model.predict_proba(X_validate, batch_size = 128, verbose = 0)
acc = roc_auc_score(y_validate, pred_auc)
print('AUC:', acc)

from sklearn.model_selection import train_test_split
id_code_test=test[index_variable]
X_normalized_test=normalize_data(test[numerical_variables])
y_pred =model.predict_proba(X_normalized_test, batch_size = 128, verbose = 0)
y_pred_rounded=np.around(y_pred)
## submission
subs = pd.DataFrame(index=test.index)
subs['ID_code'] = test['ID_code']
subs['target'] = y_pred_rounded.astype(np.int64)
subs.to_csv('submission.csv', index=False,float_format='%.0f')

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from scipy.special import logit

train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
features = [x for x in train_df.columns if x.startswith("var")]

hist_df = pd.DataFrame()
for var in features:
    var_stats = train_df[var].append(test_df[var]).value_counts()
    hist_df[var] = pd.Series(test_df[var]).map(var_stats)
    hist_df[var] = hist_df[var] > 1

ind = hist_df.sum(axis=1) != 200
var_stats = {var:train_df[var].append(test_df[ind][var]).value_counts() for var in features}

pred = 0
for var in features:
    """
    model = lgb.LGBMClassifier(**{ 'learning_rate': 0.04, 'num_leaves': 31, 'max_bin': 1023, 'min_child_samples': 1000, 'reg_alpha': 0.1, 'reg_lambda': 0.2,
     'feature_fraction': 1.0, 'bagging_freq': 1, 'bagging_fraction': 0.85, 'objective': 'binary', 'n_jobs': -1, 'n_estimators':200,})
    """
    
    model = lgb.LGBMClassifier(**{ 'learning_rate':0.05, 'max_bin': 165, 'max_depth': 5, 'min_child_samples': 150,
        'min_child_weight': 0.1, 'min_split_gain': 0.0018, 'n_estimators': 41, 'num_leaves': 6, 'reg_alpha': 2.0,
        'reg_lambda': 2.54, 'objective': 'binary', 'n_jobs': -1})
        
    model = model.fit(np.hstack([train_df[var].values.reshape(-1,1),
                                 train_df[var].map(var_stats[var]).values.reshape(-1,1)]),
                               train_df["target"].values)
    pred += logit(model.predict_proba(np.hstack([test_df[var].values.reshape(-1,1),
                                 test_df[var].map(var_stats[var]).values.reshape(-1,1)]))[:,1])
    
pd.DataFrame({"ID_code":test_df["ID_code"], "target":pred}).to_csv("submission.csv", index=False)
